In [ ]:
date_range = snakemake.params.date_range
year = snakemake.wildcards.year
column_names_dict = snakemake.params.column_names_dict
regions_dict = snakemake.params.regions_dict

date_range = [year + "-" + date for date in date_range]

In [ ]:
import logging

import atlite

logging.basicConfig(level=logging.INFO)


import geopandas as gpd
import pandas as pd
import xarray as xr
from shapely.geometry import Polygon

In [ ]:
atlite.__version__

In [ ]:
xr.__version__

In [ ]:
%psource atlite.Cutout.convert_and_aggregate

# Desired regions


In [ ]:
desired_regions = snakemake.params.regions
desired_regions.sort()
desired_regions

## Shape level

To get the desired data at shape level, the first input is the shapefile itself.


In [ ]:
regions_dict

In [ ]:
europe = (
    gpd
    .read_file(snakemake.input.euroshape)
    .replace({"GB": "UK", "EL": "GR"})
    .replace(regions_dict)
    .query("id == @desired_regions")
    .rename(columns={"id" : "index"})
    .loc[:, ["index", "geometry"]]
    .set_index(["index"])
)

In [ ]:
europe

In [ ]:
column_name = column_names_dict["column_name"]#.get_values("id")

In [ ]:
europe = (
    gpd
    .read_file(snakemake.input.euroshape)
    .replace({"GB": "UK", "EL": "GR"})
    .replace(regions_dict)
    #.rename(columns={str(column_name): "index"})
    #.loc[:, ["index", "CNTR_CODE", "geometry"]]
    #.set_index(["index"])
)

europe = (
    europe.loc[lambda europe: europe[column_name].isin(desired_regions)]
    .rename(columns={str(column_name): "index"})
    .loc[:, ["index", "geometry"]]
    .set_index(["index"])
)


In [ ]:
europe['geometry'] = europe['geometry'].buffer(0)

In [ ]:
rectx1 = -12
rectx2 = 44
recty1 = 33
recty2 = 72

polygon = Polygon(
    [
        (rectx1, recty1),
        (rectx1, recty2),
        (rectx2, recty2),
        (rectx2, recty1),
        (rectx1, recty1),
    ]
)

europe = gpd.clip(europe, polygon)

In [ ]:
europe

In [ ]:
year = snakemake.wildcards.year

In [ ]:
europe.plot(figsize=(15, 15));

In [ ]:
cutout = atlite.Cutout(path=snakemake.input.weatherdata, chunks={"time": 2000})

In [ ]:
cutout.prepare()

In [ ]:
europe = europe.sort_index()

# Hydro


In [ ]:
fn = snakemake.input.eiahydrogen

In [ ]:
# in billion KWh/a = TWh/a
eia_hydro_gen = (
    pd.read_csv(fn, skiprows=4, index_col=1, na_values=[" ", "--"])
    .drop(["Unnamed: 0", "Unnamed: 2"], axis=1)
    .dropna(how="all")
)
eia_hydro_gen

In [ ]:
import pycountry

In [ ]:
countries_iso2c = []
for country in eia_hydro_gen.index.values:
    try:
        countries_iso2c.append(pycountry.countries.get(name=country).alpha_2)
    except AttributeError:
        countries_iso2c.append(country)

In [ ]:
eia_hydro_gen.index = pd.Index(countries_iso2c, name="countries")
eia_hydro_gen

In [ ]:
eia_hydro_gen.rename(
    index={"Kosovo": "KV", "GB": "UK", "Czech Republic": "CZ", "Macedonia": "MK"},
    inplace=True,
)
eia_hydro_gen

In [ ]:
eia_hydro_gen = eia_hydro_gen.T
eia_hydro_gen

In [ ]:
eia_hydro_gen = eia_hydro_gen * 1e6  # in MWh/a
eia_hydro_gen

In [ ]:
europe.index

In [ ]:
eia_stats = eia_hydro_gen.reindex(columns=europe.index.drop_duplicates())
eia_stats

In [ ]:
eia_stats.columns

In [ ]:
europe2 = europe.dissolve(by=europe.index)

In [ ]:
europe2

In [ ]:
europe2.index.name = "countries"

In [ ]:
europe2

In [ ]:
inflow = cutout.runoff(
    shapes=europe2,
    smooth=True,
    lower_threshold_quantile=True,
    normalize_using_yearly=eia_stats,
).sel(time=slice(date_range[0], date_range[1]))

In [ ]:
inflow

In [ ]:
inflow_res = (
    inflow.assign_coords(time=range(0, len(inflow.time)))
    .assign_coords(technology="HydroRes")
    .expand_dims("technology")
    .stack(final=["time", "countries", "technology"])
    .round(2)
    .to_pandas()
    .reset_index()
)

inflow_res.columns = inflow_res.columns.map(str)
inflow_res = inflow_res.rename(columns={"0": "value"})

In [ ]:
inflow_res

In [ ]:
inflow_res.to_csv(snakemake.output["hydroresinfl"])